In [ ]:
def makeGeneratorTrumpet(gen, max_length, prev_length, pred_length):
    # inputs
    compressedSound = tf.keras.Input(shape=(20,))
    preceedingValues = tf.keras.Input(shape=(prev_length,1))
    
    # reproduce encoder until waist
    reshape =   gen.layers[6](preceedingValues)
    conc = gen.layers[8]([reshape, compressedSound])
    
    # pass through dense layers
    dense3 = gen.layers[9](conc)
    dense4 = gen.layers[10](dense3)
    dense5 = gen.layers[11](dense4)

    # output
    dense_out = gen.layers[12](dense5)
    
    model = tf.keras.Model(inputs=[compressedSound, preceedingValues], outputs=dense_out)

    # make all of it non-trainable
    for i in range(len(model.layers)):
        model.layers[i].trainable = False

    return(model) 

In [ ]:
import sys
from models.ddpm.DDPMBase import DDPMBase
from models.ddpm.CAUnet import CAUnet, UNet_conditional

import matplotlib.pyplot as plt
import torch
torch.backends.cudnn.benchmark = True

import numpy as np
import lightning as lt
torch.set_float32_matmul_precision('medium')

import soundfile as sf
from datasets import load_dataset

dataset = load_dataset('DBD-research-group/BirdSet','HSN')
file_path = dataset['test_5s'][356]['filepath']
sample, samplerate = sf.read(file_path)
sample = torch.tensor(sample,device='cuda',dtype=torch.float32)

